Realizamos los imports necesarios e iniciamos una SparkSession

In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.5` 
import $ivy.`sh.almond::almond-spark:0.4.0`

import org.apache.spark.sql.{NotebookSparkSession, SparkSession}
import org.apache.spark.sql.{functions => func, _}
import org.apache.spark.sql.types._

val spark = NotebookSparkSession
      .builder()
      .config("spark.sql.join.preferSortMergeJoin", false)
      .config("spark.sql.shuffle.partitions", 64)
      .master("local[*]")
      .getOrCreate()

import spark.implicits._

import org.slf4j.LoggerFactory
import org.apache.log4j.{Level, Logger}

Logger.getRootLogger().setLevel(Level.ERROR)

Loading spark-stubs


log4j:WARN No appenders could be found for logger (org.eclipse.jetty.util.log).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
22/09/18 19:59:21 INFO SparkContext: Running Spark version 2.4.5
22/09/18 19:59:22 INFO SparkContext: Submitted application: 8b204027-7897-4c7a-b7d3-6894add831be
22/09/18 19:59:22 INFO SecurityManager: Changing view acls to: Alvaro
22/09/18 19:59:22 INFO SecurityManager: Changing modify acls to: Alvaro
22/09/18 19:59:22 INFO SecurityManager: Changing view acls groups to: 
22/09/18 19:59:22 INFO SecurityManager: Changing modify acls groups to: 
22/09/18 19:59:22 INFO SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(Alvaro); groups with view permissions: Set(); users  with modify permissions: Set(Alvaro); groups with modify permissions: Set()
22/09/18 19:59:23 INFO Utils: Successfully started service 'sparkDriver' on port 49671.
22/09/18 19:59:23 INFO SparkEnv: Registering MapOutputTracker
22/09/18 19:59:23 INFO SparkEnv: Registering BlockManagerMa

22/09/18 19:59:23 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppData/Local/Coursier/cache/v1/https/repo1.maven.org/maven2/io/get-coursier/coursier-cache_2.12/1.1.0-M13-1/coursier-cache_2.12-1.1.0-M13-1.jar at spark://LAPTOP-30N4Q54J:49671/jars/coursier-cache_2.12-1.1.0-M13-1.jar with timestamp 1663523963746
22/09/18 19:59:23 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppData/Local/Coursier/cache/v1/https/repo1.maven.org/maven2/io/get-coursier/coursier-core_2.12/1.1.0-M13-1/coursier-core_2.12-1.1.0-M13-1.jar at spark://LAPTOP-30N4Q54J:49671/jars/coursier-core_2.12-1.1.0-M13-1.jar with timestamp 1663523963747
22/09/18 19:59:23 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppData/Local/Coursier/cache/v1/https/repo1.maven.org/maven2/io/get-coursier/coursier_2.12/1.1.0-M13-1/coursier_2.12-1.1.0-M13-1.jar at spark://LAPTOP-30N4Q54J:49671/jars/coursier_2.12-1.1.0-M13-1.jar with timestamp 1663523963748
22/09/18 19:59:23 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppDa

...

22/09/18 19:59:23 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppData/Local/Coursier/cache/v1/https/repo1.maven.org/maven2/javax/xml/stream/stax-api/1.0-2/stax-api-1.0-2.jar at spark://LAPTOP-30N4Q54J:49671/jars/stax-api-1.0-2.jar with timestamp 1663523963884
22/09/18 19:59:23 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppData/Local/Coursier/cache/v1/https/repo1.maven.org/maven2/org/roaringbitmap/RoaringBitmap/0.7.45/RoaringBitmap-0.7.45.jar at spark://LAPTOP-30N4Q54J:49671/jars/RoaringBitmap-0.7.45.jar with timestamp 1663523963887
22/09/18 19:59:23 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppData/Local/Coursier/cache/v1/https/repo1.maven.org/maven2/org/codehaus/jackson/jackson-jaxrs/1.9.13/jackson-jaxrs-1.9.13.jar at spark://LAPTOP-30N4Q54J:49671/jars/jackson-jaxrs-1.9.13.jar with timestamp 1663523963889
22/09/18 19:59:23 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppData/Local/Coursier/cache/v1/https/repo1.maven.org/maven2/io/dropwizard/metrics/metrics-js

22/09/18 19:59:23 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppData/Local/Coursier/cache/v1/https/repo1.maven.org/maven2/org/apache/arrow/arrow-memory/0.10.0/arrow-memory-0.10.0.jar at spark://LAPTOP-30N4Q54J:49671/jars/arrow-memory-0.10.0.jar with timestamp 1663523963912
22/09/18 19:59:23 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppData/Local/Coursier/cache/v1/https/repo1.maven.org/maven2/org/apache/hadoop/hadoop-mapreduce-client-shuffle/2.6.5/hadoop-mapreduce-client-shuffle-2.6.5.jar at spark://LAPTOP-30N4Q54J:49671/jars/hadoop-mapreduce-client-shuffle-2.6.5.jar with timestamp 1663523963913
22/09/18 19:59:23 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppData/Local/Coursier/cache/v1/https/repo1.maven.org/maven2/com/fasterxml/jackson/module/jackson-module-paranamer/2.7.9/jackson-module-paranamer-2.7.9.jar at spark://LAPTOP-30N4Q54J:49671/jars/jackson-module-paranamer-2.7.9.jar with timestamp 1663523963914
22/09/18 19:59:23 INFO SparkContext: Added JAR file:/C:/U

22/09/18 19:59:23 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppData/Local/Coursier/cache/v1/https/repo1.maven.org/maven2/org/eclipse/jetty/jetty-util/9.4.15.v20190215/jetty-util-9.4.15.v20190215.jar at spark://LAPTOP-30N4Q54J:49671/jars/jetty-util-9.4.15.v20190215.jar with timestamp 1663523963934
22/09/18 19:59:23 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppData/Local/Coursier/cache/v1/https/repo1.maven.org/maven2/com/chuusai/shapeless_2.12/2.3.3/shapeless_2.12-2.3.3.jar at spark://LAPTOP-30N4Q54J:49671/jars/shapeless_2.12-2.3.3.jar with timestamp 1663523963935
22/09/18 19:59:23 INFO SparkContext: Added JAR file:/C:/Users/alvar/AppData/Local/Coursier/cache/v1/https/repo1.maven.org/maven2/sh/almond/spark-stubs_24_2.12/0.4.0/spark-stubs_24_2.12-0.4.0.jar at spark://LAPTOP-30N4Q54J:49671/jars/spark-stubs_24_2.12-0.4.0.jar with timestamp 1663523963936
22/09/18 19:59:24 INFO Executor: Starting executor ID driver on host localhost
22/09/18 19:59:24 INFO Executor: Using REPL 

Spark UI

import $ivy.$                                   

import $ivy.$                              


import org.apache.spark.sql.{NotebookSparkSession, SparkSession}

import org.apache.spark.sql.{functions => func, _}

import org.apache.spark.sql.types._


spark: SparkSession = org.apache.spark.sql.SparkSession@39f4aafd
import spark.implicits._


import org.slf4j.LoggerFactory

import org.apache.log4j.{Level, Logger}



In [ ]:
val data: DataFrame = spark.read.option("multiline", "true").json("D:/TFGAlvaroSanchez/data2/*.json")

In [ ]:
data.show()

### Obtencion de la fecha y del valor maximo de temperatura

Con lo siguiente cambiamos el tipo de datos de la columna ta_max

val data1 = data.withColumn("ta_max", $"ta_max".substr(0,4).cast(IntegerType))

In [ ]:
import org.apache.spark.sql.functions.{min, max, desc}

data.withColumn("ta_max", $"ta_max".substr(0,4).cast(IntegerType))
    .select($"fecha", $"ta_max" as "temperatura maxima")
    .orderBy($"ta_max".desc)
    .show(1)

### Obtencion del mes, año, id estacion y provincia donde ocurren la stemperaturas mas altas en 2021

In [ ]:
val data2021 : DataFrame = spark.read.option("multiline", "true").json("D:/TFGAlvaroSanchez/data2/*2021.json")

In [ ]:
data2021.printSchema()

In [ ]:
data2021.select($"ta_min", $"indicativo", $"fecha")
    .filter($"indicativo".equalTo("2916A"))
    .show()

Usamos `.df` para darle nombre a las columnas

In [ ]:
val ids : DataFrame = spark.read.option("delimiter", ";").csv("D:/TFGAlvaroSanchez/data2/aemetID.csv")
    .toDF("provincia", "indicativo", "ubicacion")

In [ ]:
ids.show()
ids.printSchema()

In [ ]:
val joindata : DataFrame = data2021.join(ids, "indicativo")

In [ ]:
joindata.withColumn("ta_max", $"ta_max".substr(0,4).cast(IntegerType))
    .select($"fecha".substr(6,2) as "mes", $"fecha".substr(0,4) as "año", $"ta_max" as "temperatura maxima", $"indicativo", $"provincia")
    .filter(!($"mes".equalTo("13")))
    .orderBy($"ta_max".desc)
    .show()

### Obtencion del mes, año y provincia donde se encuntra la temperatura mas baja durante verano (junio, julio y agosto)

In [ ]:
joindata.withColumn("ta_min", $"ta_min".substr(0,4).cast(IntegerType))
    .withColumn("fecha", $"fecha".cast(DateType)).printSchema

Para quedarnos unicamente con la temperatura ya que este valor viene precedido del dia que ocurrio de la siguiente manera: temperatura(dia)

In [ ]:
import org.apache.spark.sql.functions.split

joindata.withColumn("ta_min", split($"ta_min", "\\("))
    .select($"ta_min"(0) as "val")
    .show()

In [ ]:
joindata.withColumn("ta_min", split($"ta_min", "\\("))
    .withColumn("ta_min", $"ta_min"(0).cast(IntegerType))
    .withColumn("fecha", $"fecha".cast(DateType))
    .select(func.month($"fecha") as "mes", func.year($"fecha") as "año", $"ta_min" as "temperatura minima", $"provincia")  
    .filter($"mes" > 5 && $"mes" < 9)
    .orderBy($"temperatura minima".asc)
    .show()

### Numero de meses con temperaturas maximas >30º por provincia

In [ ]:
joindata
    .withColumn("ta_max", split($"ta_max", "\\("))
    .withColumn("ta_max", $"ta_max"(0).cast(IntegerType))
    .withColumn("fecha", $"fecha".cast(DateType))
    .filter($"ta_max" > 30 && !func.isnull($"fecha"))//> "0000-00-00")
    .groupBy("provincia")
    .count
    .show()

Relacion que tiene el calor con los meses de verano

In [ ]:
joindata
    .withColumn("ta_max", split($"ta_max", "\\("))
    .withColumn("ta_max", $"ta_max"(0).cast(IntegerType))
    .withColumn("fecha", $"fecha".cast(DateType))
    .filter($"ta_max" > 30 && !func.isnull($"fecha"))
    .withColumn("mes", func.month($"fecha"))
    .stat.corr("ta_max", "mes")

### Uso de graficos mostrando la precipitacion mensual por meses y años en Vitigudino(Salamanca)

In [ ]:
val vitData : DataFrame = spark.read.option("multiline", "true").json("D:/TFGAlvaroSanchez/data2/2916A*.json")

Obtenemos los datos que queremos mostrar

In [ ]:
val vitDataRequired : DataFrame = vitData
    .withColumn("fecha", $"fecha".cast(DateType))
    .withColumn("p_mes", $"p_mes".cast(IntegerType))
    .filter(!func.isnull($"fecha") && $"p_mes" >= 0)
    .select($"fecha", $"p_mes")
    .orderBy($"fecha".asc)

In [ ]:
vitDataRequired.show()

In [ ]:
val vitDataRequired_f : Seq[String] = vitDataRequired.select($"fecha").as[String].collect.toSeq
val vitDataRequired_p : Seq[Int] = vitDataRequired.select($"p_mes").as[Int].collect.toSeq

Realizamos los imports necesarios para las graficas

In [ ]:
import $ivy.`org.plotly-scala::plotly-almond:0.7.0`
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

// restrict the output height to avoid scrolling in output cells
repl.pprinter() = repl.pprinter().copy(defaultHeight = 3)

In [ ]:
val trace = Seq(
    Scatter(
        vitDataRequired_f,
        vitDataRequired_p,
        fill = Fill.ToZeroY,
        marker = Marker(
            color = Color.RGBA(55, 128, 191, 0.6)
        )
    )
)


val layout = Layout(
    title = "Precipitacion mensual (mm)",
    paper_bgcolor =  Color.RGBA(245, 246, 249, 1),
    plot_bgcolor = Color.RGBA(245, 246, 249, 1),
)
plot(trace, layout)

### Leer los datos de manera casteada mediante un schema

In [ ]:
val schema = StructType(
                Array(
                    /*StructField("fecha", DateType, true),
                    StructField("indicativo", StringType, true),
                    StructField("p_max", StringType, true),*/
                    StructField("glo", IntegerType, true),
                    /*StructField("hr", IntegerType, true),
                    StructField("nw_55", IntegerType, true),
                    StructField("tm_min", DoubleType, true),
                    StructField("ta_max", StringType, true),
                    StructField("ts_min", DoubleType, true),
                    StructField("nt_30", IntegerType, true),
                    StructField("n_des", IntegerType, true),
                    StructField("w_racha", StringType, true),
                    StructField("np_100", IntegerType, true)*/
                )
            )

In [ ]:
val dt: DataFrame = spark.read.schema(schema)
    .option("multiline", "true")
    .option("dateFormat", "yyyy-MM")
    .json("D:/TFGAlvaroSanchez/data2/0201D(Barcelona)-2021.json")

In [ ]:
dt.show()

In [ ]:
val dt: DataFrame = spark.read
    .option("multiline", "true")
    .json("D:/TFGAlvaroSanchez/data2/0201D(Barcelona)-2021.json")
    .select(//$"fecha".cast(DateType), 
            func.to_date($"fecha").alias("fecha"),
            $"indicativo", 
            $"p_max",
            $"glo".cast(DoubleType), 
            $"hr".cast(DoubleType), 
            $"nw_55".cast(IntegerType), 
            $"tm_min".cast(DoubleType), 
            $"ta_max", 
            $"ts_min".cast(DoubleType), 
            $"nt_30".cast(IntegerType), 
            $"n_des".cast(IntegerType), 
            $"w_racha", 
            $"np_100".cast(IntegerType), 
            $"nw_91".cast(IntegerType), 
            $"np_001".cast(IntegerType), 
            $"ta_min", 
            $"w_rec".cast(IntegerType), 
            $"e".cast(DoubleType), 
            $"np_300".cast(IntegerType), 
            $"p_mes".cast(DoubleType), 
            $"w_med".cast(DoubleType), 
            $"nt_00".cast(IntegerType), 
            $"ti_max".cast(DoubleType), 
            $"tm_mes".cast(DoubleType), 
            $"tm_max".cast(DoubleType), 
            $"np_010".cast(IntegerType))

In [ ]:
dt.printSchema()

In [ ]:
dt.show()

### Temperatura media en españa

In [ ]:
val data: DataFrame = spark.read
    .option("multiline", "true")
    .json("D:/TFGAlvaroSanchez/data2/*.json")
    .select($"fecha".cast(DateType), 
            $"indicativo", 
            $"p_max",
            $"glo".cast(DoubleType), 
            $"hr".cast(DoubleType), 
            $"nw_55".cast(IntegerType), 
            $"tm_min".cast(DoubleType), 
            $"ta_max", 
            $"ts_min".cast(DoubleType), 
            $"nt_30".cast(IntegerType), 
            $"n_des".cast(IntegerType), 
            $"w_racha", 
            $"np_100".cast(IntegerType), 
            $"nw_91".cast(IntegerType), 
            $"np_001".cast(IntegerType), 
            $"ta_min", 
            $"w_rec".cast(IntegerType), 
            $"e".cast(DoubleType), 
            $"np_300".cast(IntegerType), 
            $"p_mes".cast(DoubleType), 
            $"w_med".cast(DoubleType), 
            $"nt_00".cast(IntegerType), 
            $"ti_max".cast(DoubleType), 
            $"tm_mes".cast(DoubleType), 
            $"tm_max".cast(DoubleType), 
            $"np_010".cast(IntegerType))

In [ ]:
data.filter($"fecha".isNotNull)
    .select($"tm_mes".alias("temperatura media"))
    .describe()
    .show()

### Fecha y estaciones con temp >35º (Utilizando expr)

In [ ]:
val data: DataFrame = spark.read
    .option("multiline", "true")
    .json("D:/TFGAlvaroSanchez/data2/*.json")
    .select($"fecha".cast(DateType), 
            $"indicativo", 
            $"p_max",
            $"glo".cast(DoubleType), 
            $"hr".cast(DoubleType), 
            $"nw_55".cast(IntegerType), 
            $"tm_min".cast(DoubleType), 
            $"ta_max", 
            $"ts_min".cast(DoubleType), 
            $"nt_30".cast(IntegerType), 
            $"n_des".cast(IntegerType), 
            $"w_racha", 
            $"np_100".cast(IntegerType), 
            $"nw_91".cast(IntegerType), 
            $"np_001".cast(IntegerType), 
            $"ta_min", 
            $"w_rec".cast(IntegerType), 
            $"e".cast(DoubleType), 
            $"np_300".cast(IntegerType), 
            $"p_mes".cast(DoubleType), 
            $"w_med".cast(DoubleType), 
            $"nt_00".cast(IntegerType), 
            $"ti_max".cast(DoubleType), 
            $"tm_mes".cast(DoubleType), 
            $"tm_max".cast(DoubleType), 
            $"np_010".cast(IntegerType))
val dataWithNameStation = data.join(ids, "indicativo")

In [ ]:
dataWithNameStation.withColumn("ta_max", split($"ta_max", "\\("))
    .withColumn("t_max", $"ta_max"(0).cast(IntegerType))
    .withColumn("dia", func.substring($"ta_max"(1), 0, 2).cast(IntegerType))
    .withColumn("calor", func.expr("t_max > 35"))
    .filter($"calor" && $"fecha".isNotNull)
    .select($"ubicacion", func.year($"fecha").alias("año"), func.month($"fecha").alias("mes"), $"dia")
    .show(false)

### Media de todos los datos, agrupados por fecha y por indicativo

In [ ]:
val data2021: DataFrame = spark.read
    .option("multiline", "true")
    .json("D:/TFGAlvaroSanchez/data2/*-2021.json")
    .select($"fecha".cast(DateType), 
            $"indicativo", 
            $"p_max",
            $"glo".cast(DoubleType), 
            $"hr".cast(DoubleType), 
            $"nw_55".cast(IntegerType), 
            $"tm_min".cast(DoubleType), 
            $"ta_max", 
            $"ts_min".cast(DoubleType), 
            $"nt_30".cast(IntegerType), 
            $"n_des".cast(IntegerType), 
            $"w_racha", 
            $"np_100".cast(IntegerType), 
            $"nw_91".cast(IntegerType), 
            $"np_001".cast(IntegerType), 
            $"ta_min", 
            $"w_rec".cast(IntegerType), 
            $"e".cast(DoubleType), 
            $"np_300".cast(IntegerType), 
            $"p_mes".cast(DoubleType), 
            $"w_med".cast(DoubleType), 
            $"nt_00".cast(IntegerType), 
            $"ti_max".cast(DoubleType), 
            $"tm_mes".cast(DoubleType), 
            $"tm_max".cast(DoubleType), 
            $"np_010".cast(IntegerType))

In [ ]:
val pivotedData2021 = data2021.groupBy(func.year($"fecha")).pivot("indicativo").avg()
pivotedData2021.select("year(fecha)", "0201D_avg(tm_mes)").show()

### Leer todos los datos y transformalos a formato Parquet

In [ ]:
val allData = spark.read.option("multiline", "true").json("D:/TFGAlvaroSanchez/data/*.json").select(
            $"fecha".cast(DateType), 
            $"indicativo", 
            $"p_max",
            $"glo".cast(DoubleType), 
            $"hr".cast(DoubleType), 
            $"nw_55".cast(IntegerType), 
            $"tm_min".cast(DoubleType), 
            $"ta_max", 
            $"ts_min".cast(DoubleType), 
            $"nt_30".cast(IntegerType), 
            $"n_des".cast(IntegerType), 
            $"w_racha", 
            $"np_100".cast(IntegerType), 
            $"nw_91".cast(IntegerType), 
            $"np_001".cast(IntegerType), 
            $"ta_min", 
            $"w_rec".cast(IntegerType), 
            $"e".cast(DoubleType), 
            $"np_300".cast(IntegerType), 
            $"p_mes".cast(DoubleType), 
            $"w_med".cast(DoubleType), 
            $"nt_00".cast(IntegerType), 
            $"ti_max".cast(DoubleType), 
            $"tm_mes".cast(DoubleType), 
            $"tm_max".cast(DoubleType), 
            $"np_010".cast(IntegerType))

In [ ]:
allData.write.format("parquet").mode("overwrite").save("D:/TFGAlvaroSanchez/dataParquet/")

### Representacion de las temperaturas máximas en diferentes meses por años (desde 2015)

In [14]:
import $ivy.`org.plotly-scala::plotly-almond:0.7.0`
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

// restrict the output height to avoid scrolling in output cells
repl.pprinter() = repl.pprinter().copy(defaultHeight = 3)

import $ivy.$                                      

import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

// restrict the output height to avoid scrolling in output cells


In [15]:
val dataParquet = spark.read.parquet("D:/TFGAlvaroSanchez/dataParquet/*")

val data = dataParquet
    .na.drop()
    .withColumn("ta_max", func.split($"ta_max", "\\(")(0).cast(IntegerType))
    .groupBy($"fecha")
    .agg(func.avg($"ta_max").alias("ta_max"))

parquet at cmd14.sc:1

105 / 105

parquet at cmd14.sc:1

1 / 1

dataParquet: DataFrame = [fecha: date, indicativo: string ... 24 more fields]
data: DataFrame = [fecha: date, ta_max: double]

In [16]:
val months = data
    .select(func.month($"fecha").alias("mes"))
    .distinct()
    .orderBy($"mes".asc)
    .as[Int].collect.toSeq

val data2015 = data
    .filter(func.year($"fecha") === 2015)
    .orderBy($"fecha".asc)
    .select($"ta_max")
    .as[Double].collect.toSeq

val data2016 = data
    .filter(func.year($"fecha") === 2016)
    .orderBy($"fecha".asc)
    .select($"ta_max")
    .as[Double].collect.toSeq

val data2017 = data
    .filter(func.year($"fecha") === 2017)
    .orderBy($"fecha".asc)
    .select($"ta_max")
    .as[Double].collect.toSeq

val data2018 = data
    .filter(func.year($"fecha") === 2018)
    .orderBy($"fecha".asc)
    .select($"ta_max")
    .as[Double].collect.toSeq

val data2019 = data
    .filter(func.year($"fecha") === 2019)
    .orderBy($"fecha".asc)
    .select($"ta_max")
    .as[Double].collect.toSeq

val data2020 = data
    .filter(func.year($"fecha") === 2020)
    .orderBy($"fecha".asc)
    .select($"ta_max")
    .as[Double].collect.toSeq

val data2021 = data
    .filter(func.year($"fecha") === 2021)
    .orderBy($"fecha".asc)
    .select($"ta_max")
    .as[Double].collect.toSeq

val data2022 = data
    .filter(func.year($"fecha") === 2022)
    .orderBy($"fecha".asc)
    .select($"ta_max")
    .as[Double].collect.toSeq

collect at cmd15.sc:5

8 / 8

collect at cmd15.sc:5

64 / 64

collect at cmd15.sc:5

...

collect at cmd15.sc:53

64 / 64

collect at cmd15.sc:53

8 / 8

months: Seq[Int] = WrappedArray(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12)
data2015: Seq[Double] = WrappedArray(
  18.96969696969697,
...
data2016: Seq[Double] = WrappedArray(
  21.114285714285714,
...
data2017: Seq[Double] = WrappedArray(
  18.823529411764707,
...
data2018: Seq[Double] = WrappedArray(
  20.323529411764707,
...
data2019: Seq[Double] = WrappedArray(
  18.142857142857142,
...
data2020: Seq[Double] = WrappedArray(
  19.5,
...
data2021: Seq[Double] = WrappedArray(
  21.307692307692307,
...
data2022: Seq[Double] = WrappedArray(
  20.620689655172413,
...

In [17]:
val year2015 = Scatter(
  months,
  data2015,
  name = "año 2015"
)

val year2016 = Scatter(
  months,
  data2016,
  name = "año 2016"
)

val year2017 = Scatter(
  months,
  data2017,
  name = "año 2017"
)

val year2018 = Scatter(
  months,
  data2018,
  name = "año 2018"
)

val year2019 = Scatter(
  months,
  data2019,
  name = "año 2019"
)

val year2020 = Scatter(
  months,
  data2020,
  name = "año 2020"
)

val year2021 = Scatter(
  months,
  data2021,
  name = "año 2021"
)

val year2022 = Scatter(
  months,
  data2022,
  name = "año 2022"
)



val dataToPlot = Seq(year2015, year2016, year2017, year2018, year2019, year2020, year2021, year2022)

plot(dataToPlot)

year2015: Scatter = Scatter(
  Some(
...
year2016: Scatter = Scatter(
  Some(
...
year2017: Scatter = Scatter(
  Some(
...
year2018: Scatter = Scatter(
  Some(
...
year2019: Scatter = Scatter(
  Some(
...
year2020: Scatter = Scatter(
  Some(
...
year2021: Scatter = Scatter(
  Some(
...
year2022: Scatter = Scatter(
  Some(
...
dataToPlot: Seq[Scatter] = List(
  Scatter(
...
res16_9: String = "plot-959ca203-82d0-4f8d-a545-4b7139b8db40"

In [18]:
import org.apache.spark.sql.expressions.Window

val window = Window.partitionBy("mes").orderBy($"ta_max".desc)

val dataWindow = data
    .select(func.month($"fecha").alias("mes"), func.year($"fecha").alias("año"), $"ta_max")
    .withColumn("dense_rank", func.dense_rank().over(window))
    .withColumn("perc_rank", func.percent_rank().over(window))

import org.apache.spark.sql.expressions.Window


window: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@c7bde65
dataWindow: DataFrame = [mes: int, año: int ... 3 more fields]

In [19]:
val year2015 = dataWindow
    .withColumn("dense_rank", func.dense_rank().over(window))
    .filter($"año" === 2015 && $"dense_rank" === 1).count()

val year2016 = dataWindow
    .withColumn("dense_rank", func.dense_rank().over(window))
    .filter($"año" === 2016 && $"dense_rank" === 1).count()

val year2017 = dataWindow
    .withColumn("dense_rank", func.dense_rank().over(window))
    .filter($"año" === 2017 && $"dense_rank" === 1).count()

val year2018 = dataWindow
    .withColumn("dense_rank", func.dense_rank().over(window))
    .filter($"año" === 2018 && $"dense_rank" === 1).count()

val year2019 = dataWindow
    .withColumn("dense_rank", func.dense_rank().over(window))
    .filter($"año" === 2019 && $"dense_rank" === 1).count()

val year2020 = dataWindow
    .withColumn("dense_rank", func.dense_rank().over(window))
    .filter($"año" === 2020 && $"dense_rank" === 1).count()

val year2021 = dataWindow
    .withColumn("dense_rank", func.dense_rank().over(window))
    .filter($"año" === 2021 && $"dense_rank" === 1).count()

val year2022 = dataWindow
    .withColumn("dense_rank", func.dense_rank().over(window))
    .filter($"año" === 2022 && $"dense_rank" === 1).count()

count at cmd18.sc:3

8 / 8

count at cmd18.sc:3

64 / 64

count at cmd18.sc:3

...

count at cmd18.sc:31

64 / 64

count at cmd18.sc:31

1 / 1

year2015: Long = 1L
year2016: Long = 1L
year2017: Long = 2L
year2018: Long = 0L
year2019: Long = 0L
year2020: Long = 2L
year2021: Long = 2L
year2022: Long = 2L

In [20]:
dataWindow
    .filter($"dense_rank" === 1)
    .withColumn("temperatura maxima", func.round($"ta_max", 2))
    .orderBy($"mes".asc)
    .select($"mes", $"año", $"temperatura maxima")
    .show()

val dataToPlot = Seq(
  Bar(
    Seq("2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022"),
    Seq(year2015, year2016, year2017, year2018, year2019, year2020, year2021, year2022),
    marker = Marker(
        color = Color.RGB(255, 174, 0),
        opacity = 0.6,
        line = Line(
            color = Color.RGB(189, 129, 0),
              width = 1.5
        )
    )
  )
)

plot(dataToPlot)

show at cmd19.sc:5

8 / 8

show at cmd19.sc:5

64 / 64

show at cmd19.sc:5

64 / 64

+---+----+------------------+
|mes| año|temperatura maxima|
+---+----+------------------+
|  1|2021|             21.31|
|  2|2020|             23.96|
|  3|2017|              27.3|
|  4|2011|             29.13|
|  5|2015|              33.3|
|  6|2017|             36.53|
|  7|2022|             39.07|
|  8|2022|              37.2|
|  9|2016|              36.0|
| 10|2011|             31.09|
| 11|2020|              25.0|
| 12|2021|             21.86|
+---+----+------------------+



dataToPlot: Seq[Bar] = List(
  Bar(
...
res19_2: String = "plot-4c62adee-6f7a-4f15-9016-28ad8425da84"

### Años/meses con olas de calor (30º durante >= 3 dias)

In [2]:
import $ivy.`org.plotly-scala::plotly-almond:0.7.0`
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

// restrict the output height to avoid scrolling in output cells
repl.pprinter() = repl.pprinter().copy(defaultHeight = 3)

import $ivy.$                                      

import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

// restrict the output height to avoid scrolling in output cells


In [3]:
val data = spark.read.parquet("D:/TFGAlvaroSanchez/dataParquet/*").na.drop()

parquet at cmd2.sc:1

105 / 105

parquet at cmd2.sc:1

1 / 1

data: DataFrame = [fecha: date, indicativo: string ... 24 more fields]

In [4]:
val groupData = data
    .filter($"nt_30" >= 3)
    .withColumn("año", func.year($"fecha"))
    .withColumn("ta_max", func.split($"ta_max", "\\(")(0).cast(IntegerType))
    .groupBy($"año")
    .agg(func.avg($"nt_30").alias("nt_30"), func.avg($"ta_max").alias("ta_max"))
    .select($"año", func.round($"nt_30", 2) as "dias al mes con >=30º", func.round($"ta_max", 2) as "temperatura media max")
    .orderBy($"año")

groupData.show()

show at cmd3.sc:10

8 / 8

show at cmd3.sc:10

64 / 64

+----+---------------------+---------------------+
| año|dias al mes con >=30º|temperatura media max|
+----+---------------------+---------------------+
|2010|                13.61|                35.58|
|2011|                12.96|                34.95|
|2012|                13.12|                35.77|
|2013|                13.93|                34.61|
|2014|                13.64|                35.05|
|2015|                13.74|                35.94|
|2016|                15.04|                36.44|
|2017|                13.74|                35.98|
|2018|                16.43|                35.49|
|2019|                15.48|                36.17|
|2020|                13.19|                35.44|
|2021|                13.79|                36.01|
|2022|                16.52|                37.79|
+----+---------------------+---------------------+



groupData: Dataset[Row] = [año: int, dias al mes con >=30º: double ... 1 more field]

In [11]:
val SeqYears = groupData.map(row => row.getInt(0)).collect().toSeq
val SeqDays = groupData.map(row => row.getDouble(1)).collect().toSeq
val SeqTemperature = groupData.map(row => row.getDouble(2).toInt).collect().toSeq

collect at cmd10.sc:1

8 / 8

collect at cmd10.sc:1

64 / 64

collect at cmd10.sc:1

...

collect at cmd10.sc:3

64 / 64

collect at cmd10.sc:3

13 / 13

SeqYears: Seq[Int] = WrappedArray(
  2010,
...
SeqDays: Seq[Double] = WrappedArray(
  13.61,
...
SeqTemperature: Seq[Int] = WrappedArray(
  35,
...

In [13]:
val dataToPlot = Scatter(
    SeqYears,
    SeqDays,
    mode = ScatterMode(ScatterMode.Markers),
    marker = Marker(
        color = Seq(Color.RGB(251, 212, 68), Color.RGB(255, 223, 106), Color.RGB(251, 212, 68), Color.RGB(255, 223, 106),
                    Color.RGB(251, 212, 68), Color.RGB(251, 212, 68), Color.RGB(220, 170, 1), Color.RGB(251, 212, 68), 
                    Color.RGB(251, 212, 68), Color.RGB(220, 170, 1), Color.RGB(251, 212, 68), Color.RGB(220, 170, 1),
                    Color.RGB(220, 143, 1)),
        size = SeqTemperature
    )
)

val aux = Seq(dataToPlot)
plot(aux)

dataToPlot: Scatter = Scatter(
  Some(
...
aux: Seq[Scatter] = List(
  Scatter(
...
res12_2: String = "plot-a971c644-e3be-40b9-a165-f7d7354acd39"

### Temperatura media por año y su relacion con las lluvias por año

In [ ]:
import $ivy.`org.plotly-scala::plotly-almond:0.7.0`
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

// restrict the output height to avoid scrolling in output cells
repl.pprinter() = repl.pprinter().copy(defaultHeight = 3)

In [ ]:
val data = spark.read.parquet("D:/TFGAlvaroSanchez/dataParquet/*").na.drop()

In [ ]:
val groupData = data
    .withColumn("año", func.year($"fecha"))
    .groupBy($"año")
    .agg(func.avg($"tm_mes").alias("tm_mes"), func.avg($"p_mes").alias("p_mes"))
    .orderBy($"año")

In [ ]:
groupData.first().fieldIndex("año")
groupData.first().fieldIndex("tm_mes")
groupData.first().fieldIndex("p_mes")

In [ ]:
val SeqYears = groupData.map(row => row.getInt(0)).collect().toSeq
val SeqTemperature = groupData.map(row => row.getDouble(1)).collect().toSeq
val SeqPrecipitation = groupData.map(row => row.getDouble(2)).collect().toSeq

In [ ]:
val dataTemperature = Scatter(
    SeqYears,
    SeqTemperature,
    name = "Temperatura media (ºC)",
    marker = Marker(
        color = Color.RGB(247, 112, 0)
    )
)

val dataPrecipitation = Scatter(
    SeqYears,
    SeqPrecipitation,
    name = "Precipitacion media (mm)",
    marker = Marker(
        color = Color.RGB(0, 187, 247)
    )
)

val layout1 = Layout(
    title = "Precipitacion y temperatura media anual"
)

val layout2 = Layout(
    title = "Temperatura media anual"
)

val dataToPlot = Seq(dataTemperature, dataPrecipitation)
plot(dataToPlot, layout1)

plot(Seq(dataTemperature), layout2)